In [1]:
import subprocess
import os
import glob
from comby import Comby
from openpyxl import load_workbook
import pandas as pd

In [2]:
dm = pd.read_excel('dynamicMapping.xlsx', sheet_name = 'data' )

print( len(dm) )
print( len(dm['packageName'].unique()) )
packageNames = dm['packageName'].unique()

packageNames.sort()
print(packageNames)

10700
24
['ch.blinkenlights.android.vanilla' 'com.adam.aslfms'
 'com.amaze.filemanager' 'com.ames.books' 'com.better.alarm'
 'com.byagowi.persiancalendar' 'com.danielkim.soundrecorder'
 'com.duckduckgo.mobile.android' 'com.frozendevs.cache.cleaner'
 'com.frozendevs.periodictable' 'com.fsck.k9' 'com.github.wdkapps.fillup'
 'com.google.android.apps.authenticator2'
 'com.google.zxing.client.android'
 'com.greenaddress.greenbits_android_wallet' 'com.gsnathan.pdfviewer'
 'com.hobbyone.HashDroid' 'com.iven.musicplayergo'
 'com.redblaster.hsl.main' 'com.sagar.screenshift2' 'de.danoeh.antennapod'
 'org.torproject.torbrowser_alpha' 'org.xbmc.kodi' 'ws.xsoh.etar']


In [3]:
temp = pd.read_excel (r'shortdatasheet.xlsx')

mapping = {}

temp.sort_values(by='Package')

for index, row in temp.iterrows():
    if( row['Package'] in packageNames):
        mapping[row['Package']] = row['Name']
        print(row['Package'])
        
print(os.getcwd())
print( mapping )

ch.blinkenlights.android.vanilla
com.adam.aslfms
com.amaze.filemanager
com.ames.books
com.better.alarm
com.byagowi.persiancalendar
com.danielkim.soundrecorder
com.duckduckgo.mobile.android
com.frozendevs.cache.cleaner
com.frozendevs.periodictable
com.fsck.k9
com.github.wdkapps.fillup
com.google.android.apps.authenticator2
com.google.zxing.client.android
com.greenaddress.greenbits_android_wallet
com.gsnathan.pdfviewer
com.hobbyone.HashDroid
com.iven.musicplayergo
com.redblaster.hsl.main
com.sagar.screenshift2
org.xbmc.kodi
org.torproject.torbrowser_alpha
de.danoeh.antennapod
ws.xsoh.etar
/home/sajed/GitHub/apk-bakery/data crawler
{'ch.blinkenlights.android.vanilla': 'Vanilla Music', 'com.adam.aslfms': 'Simple Scrobbler', 'com.amaze.filemanager': 'Amaze', 'com.ames.books': 'BookList', 'com.better.alarm': 'Simple Alarm Clock', 'com.byagowi.persiancalendar': 'Persian Calendar', 'com.danielkim.soundrecorder': 'Sound Recorder', 'com.duckduckgo.mobile.android': 'DuckDuckGo Privacy Browser', '

In [4]:
class Filepaths:
    
    def __init__(self, kotlin, java):
        self.kotlin = kotlin
        self.java = java

In [5]:

filePathDict = {}

for key, value in mapping.items():
    # print(key, value)
    
    kotlin = []
    java = []
    
    for x in os.walk( os.path.join(os.getcwd(), 'code', value ) ):
        for y in glob.glob(os.path.join(x[0], '*.kt')):
            kotlin.append(y)
    for x in os.walk( os.path.join(os.getcwd(), 'code', value) ):
        for y in glob.glob(os.path.join(x[0], '*.java')):
            java.append(y)
            
    print(len(kotlin), len(java))
    
    filePathDict[ value ] = Filepaths(kotlin, java)

# print(filePathDict)

0 113
0 87
266 266
0 34
83 21
163 0
0 14
1412 8
0 10
0 28
716 440
0 58
0 99
0 513
282 56
0 9
0 21
42 0
0 50
0 38
0 13
8 55
0 528
17 101


In [6]:
fileContentDict = {}

for folder_name, filePaths_Obj in filePathDict.items():
    
    kotlin = filePaths_Obj.kotlin
    java = filePaths_Obj.java
    
    # print(kotlin, java)
    
    for file in kotlin:
        fileContentDict[file] = open(file, "r").read()
    for file in java:
        fileContentDict[file] = open(file, "r").read()

print( len(fileContentDict))

5551


In [7]:
def formatResourceId(resourceId: str):
    words = resourceId.split('_')
    result: str = ""
    for i in enumerate(words):
        if( i[0] == 0 ):
            result = result + i[1]
        else:
            result = result + i[1].capitalize()
    
    return result

In [8]:
cacheDict = {}

In [9]:
def getCombyResult(row, filePaths: Filepaths, resource):
    
    resourceId = formatResourceId(resource)
    
    
    cacheCheck = mapping[package] + " " + row['resourceIds'].split('/')[1]
    
                
    if cacheCheck in cacheDict:
        return cacheDict[cacheCheck]
    
    data = []
    comby = Comby()
    
    for file in filePaths.kotlin:
        objs = list(comby.matches( fileContentDict[file], resourceId))
        if objs:
            for obj in objs:
                # print(data.location)
                temp = []
                temp.extend([file, obj.location])
                data.append( temp )
                
    for file in filePaths.java:
        objs = list(comby.matches( fileContentDict[file], resourceId))
        if objs:
            for obj in objs:
                # print(data.location)
                temp = []
                temp.extend([file, obj.location])
                data.append( temp )
    # print(data)
    
    cacheDict[cacheCheck] = data
    
    return data

In [10]:
finalResult = []

found = 0;
notFound = 0;
exception = 0

print(len(dm))

# print(dm)


cacheDict = {}
for index, row in dm.iterrows():

    package: str = row['packageName'].split('-')[0]

    if package in mapping.keys():
        # print(mapping[package], row['resourceIds'].split('/')[1])
        try:

            result_line = []

            results = getCombyResult( row, filePathDict[ mapping[package] ] , row['resourceIds'].split('/')[1] )
            for result in results:
                temp = row.tolist()
                temp.extend(result)
                result_line.append(temp)


            if not pd.DataFrame(result_line).empty :
                finalResult.extend(result_line)
                print('Done for row: '+ f'{index}' )
                found = found + 1
            else:
                print('Not Found for row: '+ f'{index}' )
                notFound = notFound + 1
        except Exception:
            print("An exception occurred for row: "+ f'{index}')
            exception = exception + 1
        

10700
Not Found for row: 0
Done for row: 1
Done for row: 2
Done for row: 3
Done for row: 4
Done for row: 5
Done for row: 6
Done for row: 7
Done for row: 8
Done for row: 9
Done for row: 10
Done for row: 11
Done for row: 12
Done for row: 13
Done for row: 14
Not Found for row: 15
Not Found for row: 16
Not Found for row: 17
Done for row: 18
Done for row: 19
Done for row: 20
Done for row: 21
Done for row: 22
Done for row: 23
Done for row: 24
Not Found for row: 25
Done for row: 26
Done for row: 27
Done for row: 28
Done for row: 29
Done for row: 30
Not Found for row: 31
Not Found for row: 32
Not Found for row: 33
Not Found for row: 34
Not Found for row: 35
Not Found for row: 36
Not Found for row: 37
Not Found for row: 38
Not Found for row: 39
Not Found for row: 40
Not Found for row: 41
Not Found for row: 42
Not Found for row: 43
Done for row: 44
Not Found for row: 45
Not Found for row: 46
Done for row: 47
Done for row: 48
Done for row: 49
Done for row: 50
Done for row: 51
Done for row: 52
Don

In [11]:
finaldf = pd.DataFrame(finalResult)
finaldf.to_excel( 'FinalDataset-test0.xlsx', sheet_name='data', index = False)

In [12]:
print(found, notFound, exception)

5944 4699 57


In [13]:
cacheDict = { "Hello": 56,
    "at": 23,
    "test": 43,
    "this": 78 }

if 'tt' in cacheDict:
    print( cacheDict["this"] )
else:
    cacheDict["tt"] = 76
    
print(cacheDict)

{'Hello': 56, 'at': 23, 'test': 43, 'this': 78, 'tt': 76}


In [14]:
len(finaldf)

548039